In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from scipy.stats import pearsonr

In [4]:
BASE_DIR = '/sharedHDD/rohit/timeseries_learning/ubiquant/'
OUTPUT_DIR = BASE_DIR+'output/'

In [5]:
# model_name = 'tabnet'
# target_o = 'target'
# target_p = 'predicted'

model_name = 'lstm4'
target_o = 'target_o'
target_p = 'target'

In [6]:
def weighted_average(a):
    w = []
    n = len(a)
    for j in range(1, n + 1):
        j = 2 if j == 1 else j
        w.append(1 / (2**(n + 1 - j)))
    return np.average(a, weights = w)

In [7]:
def get_pearson_r(row):
    v,_ = pearsonr(row.target_p,row.target_o)
    return v

In [8]:
scores = []
for fold in [0,1,2,3,4]:
    df = pd.read_csv(OUTPUT_DIR+str(fold)+'_'+model_name+'.csv')
    grp_df = df.groupby('time_id',as_index=False).agg(target_o = (target_o, list),
                                                  target_p = (target_p, list),
                                                  num_points = (target_p, len)
                                                 )
    grp_df = grp_df[grp_df.num_points>=2]
    grp_df['pearson_r'] = grp_df.apply(lambda x: get_pearson_r(x), axis=1)
    s = grp_df.pearson_r.mean()
    scores.append(s)
    print(f'Score for fold {fold} is {s}')

Score for fold 0 is 0.13872537344416577
Score for fold 1 is 0.1403345231387604
Score for fold 2 is 0.14464161185040084
Score for fold 3 is 0.13782713654464257
Score for fold 4 is 0.12343853328101567


In [9]:
weighted_average(scores)

0.1316974957944015